# <a id='toc1_'></a>[2. Guidelines and Principles for prompting](#toc0_)

**Table of contents**<a id='toc0_'></a>
- [Principle 1: write clear and specific instructions](#toc1_1_)    
    - [Tactic 1: use delimiters to indicate distinct parts of the input.](#toc1_1_1_1_)    
    - [Tactic 2: Ask for structured output.](#toc1_1_1_2_)    
    - [Tactic 3: Check whether conditions are met. Check assumptions and preconditions required to do the task.](#toc1_1_1_3_)    
    - [Tactic 4: "Few-shot" prompts.](#toc1_1_1_4_)    
- [Principle 2: give the model "time to think"](#toc1_2_)    
    - [Tactic 1: Specify the steps required to complete the task.](#toc1_2_1_1_)    
    - [Tactic 2: Ask the model to work out its solution before giving the final answer.](#toc1_2_1_2_)    
- [Model limitations: hallucinations](#toc1_3_)    
    - [Tactic to reduce hallucinations: ask the model to find relevant information to answer the question, and then answer the question based on that information.](#toc1_3_1_1_)    
- [Own experiment: instructions for wrapping a gift and a tea cup](#toc1_4_)    
- [Another own experiment: analysis of Shakespeare's Sonnet 116](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [4]:
import openai
import os

from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{
        "role":"user",
        "content":prompt,
    }]

    response = openai.ChatCompletion.create(
        model=model, 
        messages=messages,
        temperature=0.0    # degree of randomness of the model's output
    )

    return response.choices[0].message['content']

## <a id='toc1_1_'></a>[Principle 1: write clear and specific instructions](#toc0_)

Clear $\neq$ Short. More clarity and context is better (more detailed, more relevant outputs).

#### <a id='toc1_1_1_1_'></a>[Tactic 1: use delimiters to indicate distinct parts of the input.](#toc0_)
- triple quotes: """ (e.g. for distinct text blocks)
- triple backticks: ``` (e.g. for distinct code blocks)
- triple dashes: --- (e.g. for distinct sections)
- angle brackets: <...> 
- xml tags: <tag>...</tag> 

In [14]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print(response)

Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.


#### <a id='toc1_1_1_2_'></a>[Tactic 2: Ask for structured output.](#toc0_)

E.g. JSON, HTML, CSV, etc.

In [15]:
prompt = f"""
Generate a list of three made-up book titles along 
with their authors and genres. \
Provide them in JSON format with the following keys: 
book_id, title, author, genre. \
Afterwards, provide them in CSV format with the 
following columns: book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

JSON format:
```
[
  {
    "book_id": 1,
    "title": "The Lost City",
    "author": "Samantha Lee",
    "genre": "Adventure"
  },
  {
    "book_id": 2,
    "title": "The Secret Garden of Dreams",
    "author": "Olivia Chen",
    "genre": "Fantasy"
  },
  {
    "book_id": 3,
    "title": "The Last Hope",
    "author": "David Kim",
    "genre": "Science Fiction"
  }
]
```

CSV format:
```
book_id,title,author,genre
1,The Lost City,Samantha Lee,Adventure
2,The Secret Garden of Dreams,Olivia Chen,Fantasy
3,The Last Hope,David Kim,Science Fiction
```


#### <a id='toc1_1_1_3_'></a>[Tactic 3: Check whether conditions are met. Check assumptions and preconditions required to do the task.](#toc0_)

In [16]:
# A paragraph describing the steps to make a cup of tea
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!


In [17]:
# A paragraph describing a beautiful day
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### <a id='toc1_1_1_4_'></a>[Tactic 4: "Few-shot" prompts.](#toc0_)

Give examples of the desired output, of successfully completed tasks, before asking the model to perform the task.

In [18]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


## <a id='toc1_2_'></a>[Principle 2: give the model "time to think"](#toc0_)

If the model gives incorrect solutions or makes reasoning errors, request a chain of reasoning or a series of steps.

#### <a id='toc1_2_1_1_'></a>[Tactic 1: Specify the steps required to complete the task.](#toc0_)

In [6]:
# A description of the story of Jack and Jill
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts. 
Noms: Jack, Jill.

{
  "french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.",
  "num_names": 2
}


Example asking for output in a specific format:

In [7]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into Italian.
3 - List each name in the Italian summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down a hill, returning home slightly battered but with undimmed adventurous spirits. 
Translation: Jack e Jill vanno in una missione per prendere acqua, ma la sfortuna colpisce e cadono giù da una collina, tornando a casa leggermente malconci ma con spirito avventuroso intatto.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack and Jill go on a quest to fetch water, but misfortune strikes and they tumble down a hill, returning home slightly battered but with undimmed adventurous spirits.", "num_names": 2}


#### <a id='toc1_2_1_2_'></a>[Tactic 2: Ask the model to work out its solution before giving the final answer.](#toc0_)

In [8]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square foot \
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
# The student's solution is not correct. The "100x" term in the
# maintenance cost should be "10x". Therefore, the total cost 
# should be 360x + 100,000.

response = get_completion(prompt)
print(response)

The student's solution is correct.


Oh but the solution is not correct. Ask the model to work out its solution first, to explain its reasoning, before giving the final answer.

In [9]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


## <a id='toc1_3_'></a>[Model limitations: hallucinations](#toc0_)

The model may make statements that sound plausible but are not true. This is due to the model being trained on a large corpus of text and knowledge, but not always perfectly memorizing the information. Therefore, it may make up information when asked about a topic it is not familiar with.

Example: Boie is a real company. The product name _AeroGlide UltraSlim Smart Toothbrush_ is not real.

In [10]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie.
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. The toothbrush features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the most stubborn stains and buildup, and it can help you achieve a brighter, healthier smile in just a few weeks.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These features include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're

#### <a id='toc1_3_1_1_'></a>[Tactic to reduce hallucinations: ask the model to find relevant information to answer the question, and then answer the question based on that information.](#toc0_)

In [12]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie. Find relevant information about the company \
and the product to support your answer. Only provide the description of the product based on the \
information you found. Cite all your sources.
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that promises to provide a superior cleaning experience. Boie is a company that specializes in creating eco-friendly personal care products that are designed to be both effective and sustainable. The AeroGlide UltraSlim Smart Toothbrush is one of their flagship products, and it features a number of advanced features that set it apart from other toothbrushes on the market.

According to the Boie website, the AeroGlide UltraSlim Smart Toothbrush is designed to be ultra-slim and lightweight, making it easy to use and maneuver. The toothbrush also features a powerful motor that delivers up to 40,000 brush strokes per minute, ensuring a thorough cleaning experience. Additionally, the toothbrush is equipped with a smart timer that helps users ensure they are brushing for the recommended two minutes.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its use of silicone bristles. Unlike traditional

It hallucinates the sources as well! This is a widely known problem of GPT-3.5. Seems to happen much less with GPT-4, which is the newest version of OpenAI's GPT model at the time of writing.

## <a id='toc1_4_'></a>[Own experiment: instructions for wrapping a gift and a tea cup](#toc0_)

I am terrible at wrapping gifts. Let's see if GPT-3.5 can help me with that.

In [13]:
prompt = f"""
Write a step-by-step guide on how to wrap a gift. \
Include a list of materials needed and the steps \
to follow, in the following format: \
    Materials: <list of materials> \
    Steps: \
    1. <step 1> \
    2. <step 2> \
    ... \
    N. <step N>
"""

response = get_completion(prompt)
print(response)

Materials:
- Wrapping paper
- Scissors
- Tape
- Ribbon or bow
- Gift tag (optional)

Steps:
1. Cut a piece of wrapping paper that is large enough to fully cover the gift. 
2. Place the gift in the center of the wrapping paper. 
3. Fold one side of the wrapping paper over the gift and tape it down. 
4. Fold the other side of the wrapping paper over the gift and tape it down. 
5. Fold one end of the wrapping paper over the gift and tape it down. 
6. Fold the other end of the wrapping paper over the gift and tape it down. 
7. If desired, tie a ribbon or bow around the gift. 
8. Attach a gift tag with the recipient's name (optional). 
9. Admire your beautifully wrapped gift!


It seems it has gone for the simplest approach, but it should work. Let's get things a bit more complex.

In [14]:
prompt = f"""
Write a step-by-step guide on how to wrap a tea cup in an \
elegant way, with the least amount of tape possible. \
Include a list of materials needed and the steps \
to follow, in the following format: \
    Materials: <list of materials> \
    Steps: \
    1. <step 1> \
    2. <step 2> \
    ... \
    N. <step N>
"""

response = get_completion(prompt)
print(response)

Materials:
- Wrapping paper
- Ribbon or twine
- Scissors
- Tape (optional)

Steps:
1. Cut a piece of wrapping paper that is large enough to cover the tea cup. 
2. Place the tea cup in the center of the wrapping paper. 
3. Bring the sides of the wrapping paper up and over the tea cup, making sure to leave enough paper to cover the top of the cup. 
4. Fold the excess paper over the top of the cup and tuck it inside. 
5. Hold the paper in place and tie a ribbon or twine around the cup, making a bow on top. 
6. If necessary, use a small piece of tape to secure the ribbon or twine in place. 
7. Adjust the bow and trim any excess ribbon or twine. 
8. Your tea cup is now elegantly wrapped and ready to be gifted.


For the sake of the experiment, of course, I've tried it out. By step 5, I already had a pretty good idea of how the thing would look like.

<img src="images/wrapped-tea-cup.jpeg" alt="" width=400px>

Maybe the draft of math exercises makes up for the lack of elegance of this wrapping.



## <a id='toc1_5_'></a>[Another own experiment: analysis of Shakespeare's Sonnet 116](#toc0_)

Let's see how GPT-3.5 does with analyzing poetry.

In [16]:
text = f"""
Let me not to the marriage of true minds
Admit impediments. Love is not love
Which alters when it alteration finds,
Or bends with the remover to remove:
O, no! it is an ever-fixed mark,
That looks on tempests and is never shaken;
It is the star to every wandering bark,
Whose worth's unknown, although his height be taken.
Love's not Time's fool, though rosy lips and cheeks
Within his bending sickle's compass come;
Love alters not with his brief hours and weeks,
But bears it out even to the edge of doom.
If this be error and upon me proved,
I never writ, nor no man ever loved.
"""

prompt = f"""
Given the Sonnet 116 by Shakespeare delimited by triple backticks, \
perform the following actions: \
1 - Analyze the metre of the poem. \
2 - Analyze the rhyme scheme of the poem. \
3 - Identify the theme of the poem. \
4 - Identify the use of poetic devices in the poem. \
5 - Summarize the poem with 1 sentence. \
6 - Analyze the adequacy of the summary. \
7 - Translate the poem into modern English. \
8 - Analyze the adequacy of the translation. \

Separate your answers with line breaks, and use the \
following format: \
1 - <answer 1> \
2 - <answer 2> \
... \
N - <answer N>

Text:
```{text}```
"""

response = get_completion(prompt)
print(response)

1 - The metre of the poem is iambic pentameter.
2 - The rhyme scheme of the poem is ABAB CDCD EFEF GG.
3 - The theme of the poem is the constancy and endurance of true love.
4 - The poem uses the poetic devices of metaphor, personification, and repetition.
5 - Love is an unchanging force that endures all obstacles and is not subject to time or change.
6 - The summary is adequate in capturing the main idea of the poem.
7 - Do not let me deny the union of two true minds; love is not love if it changes when it finds a change or bends from its firm stand even when a lover is unfaithful; love is a constant star that guides every wandering ship, whose value is beyond measure, even though its altitude can be measured; love is not subject to time, though it may be subject to the physical changes of youth and beauty, it remains constant until the end of time; if I am wrong about this, then I have never written anything, and no man has ever loved.
8 - The translation is adequate in conveying the

Pretty decent. I'd expect more detail regarding the analysis of the summary and the translation, but the translation into modern English is better than I expected.